In [ ]:
%%writefile Parallel.cu
#include <stdio.h>
#include <malloc.h>
#include <cuda.h>
#define M 100
#define Nt 20
#define dxy 0.1
#define dt 0.01
#define GridSize 10
#define BlockSize 10
#define ThreadSize M*M/(GridSize*BlockSize) 
//==============================
float funct1(int i, int j) {
  return sin(M_PI*(i*dxy + j*dxy));
}
float funct2(int i, int j) {
  return cos(M_PI*(i*dxy + j*dxy));
}
//==============================
void InputData(float *U, float *V, float *C) {
  int i,j,t;
  for (j=0; j<M; j++) {
    for (i=0; i<M; i++) {
      *(U + j*M + i) = funct1(i,j);
      *(V + j*M + i) = dt*funct2(i,j) + *(U + j*M + i);
    }
  }
  for (t=0; t<Nt; t++) {
    *(C+t) = 1/2;
  }
}
//===========================
__global__ void Computing(float *A, float *B, float *C, int t){
  float left, right, bottom, top, center1, center2;
  int l, index, start, stop;
  index = blockIdx.x * blockDim.x + threadIdx.x;
  start = index*ThreadSize;
  stop  = start + ThreadSize;
  for (l=start; l<stop; l++) {
      left  =  (l%M == 0)       ? 0 : *(B + l - 1);
      right =  (l%M == M-1)    ? 0 : *(B + l + 1);
      bottom = (l >= M*(M-1))  ? 0 : *(B + l - M);
      top   =  (l <= M-1)       ? 0 : *(B + l + M);
      center1 = *(B + l);
      center2 = *(A + l);
      *(A + l) = 2*center1 - center2 + ((dt * dt * (*(C + t)) * (*(C + t))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
    }
    __syncthreads();
}
//===========================
int main() {
  float *UCPU, *VCPU, *CCPU;
  UCPU = (float *) malloc (M*M*sizeof(float));
  VCPU = (float *) malloc (M*M*sizeof(float));
  CCPU = (float *) malloc (Nt*sizeof(float));
  InputData(UCPU, VCPU, CCPU);

  // Delare and Allocate Mem on GPU
  float *UGPU, *VGPU, *CGPU;
  cudaMalloc((void**)&UGPU ,M*M*sizeof(float));
  cudaMalloc((void**)&VGPU ,M*M*sizeof(float));
  cudaMalloc((void**)&CGPU ,Nt*sizeof(float));

  // Copy Input from CPU to GPU
  cudaMemcpy(UGPU,UCPU,M*M*sizeof(float),cudaMemcpyHostToDevice);
  cudaMemcpy(VGPU,VCPU,M*M*sizeof(float),cudaMemcpyHostToDevice);
  cudaMemcpy(CGPU,CCPU,Nt*sizeof(float),cudaMemcpyHostToDevice);

  //Define Block and Thread Structure
  dim3 dimGrid(GridSize);
  dim3 dimBlock(BlockSize);

  //Computing
  for (int t=2; t<Nt; t++) {
    if (t%2==0) Computing<<<dimGrid,dimBlock>>>(UGPU, VGPU, CGPU, t);
    else Computing<<<dimGrid,dimBlock>>>(VGPU, UGPU, CGPU, t);
  }

  //Copy Output from GPU to CPU
  cudaMemcpy(UCPU, UGPU, M*M*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(VCPU, VGPU, M*M*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(CCPU, CGPU, Nt*sizeof(float), cudaMemcpyDeviceToHost);

  // Open file for writing
  FILE *file = fopen("GPU_result.txt", "w");
  if (file == NULL) {
    printf("Error opening file!\n");
    return 1;
  }

  // Write result to file
  if (Nt % 2 == 1) {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        fprintf(file, "%f ", *(UCPU + j * M + i));
      }
      fprintf(file, "\n");
    }
  } else {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        fprintf(file, "%f ", *(VCPU + j * M + i));
      }
      fprintf(file, "\n");
    }
  }

  // Close the file
  fclose(file);

  // Free Mem on CPU and GPU
  free(UCPU); free(VCPU); free(CCPU);
  cudaFree(UGPU); cudaFree(VGPU); cudaFree(CGPU);
  return 0;
}

In [ ]:
!nvcc Parallel.cu -lm

In [ ]:
!./a.out

In [ ]:
%%writefile Sequential.cu
#include <stdio.h>
#include <malloc.h>
#include <math.h>
#define M 100
#define Nt 20
#define dxy 0.1
#define dt 0.01
//==============================
float funct1(int i, int j) {
  return sin(M_PI*(i*dxy + j*dxy));
}
float funct2(int i, int j) {
  return cos(M_PI*(i*dxy + j*dxy));
}
//==============================
void InputData(float *U, float *V, float *C) {
  int i,j,t;
  for (j=0; j<M; j++) {
    for (i=0; i<M; i++) {
      *(U + j*M + i) = funct1(i,j);
      *(V + j*M + i) = dt*funct2(i,j) + *(U + j*M + i);
    }
  }
  for (t=0; t<Nt; t++) {
    *(C+t) = 1/2;
  }
}
//===============================
void Computing(float *A, float *B, float *C, int t) {
  int l;
  float left, right, bottom, top, center1, center2;
  for (l=0; l<M*M; l++) {
      left  =  (l%M == 0)       ? 0 : *(B + l - 1);
      right =  (l%M == M-1)    ? 0 : *(B + l + 1);
      bottom = (l >= M*(M-1))  ? 0 : *(B + l - M);
      top   =  (l <= M-1)       ? 0 : *(B + l + M);
      center1 = *(B + l);
      center2 = *(A + l);
      *(A + l) = 2*center1 - center2 + ((dt * dt * (*(C + t)) * (*(C + t))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
  }
}
//===============================
int main() {
  float *U, *V, *C;
  int t;
  U = (float *) malloc (M*M*sizeof(float));
  V = (float *) malloc (M*M*sizeof(float));
  C = (float *) malloc (Nt*sizeof(float));
  InputData(U, V, C);
  for (t=2; t<Nt; t++) {
    if (t%2==0) Computing(U, V, C, t);
    else Computing(V, U, C, t);
  }
    
  FILE *file = fopen("CPU_result.txt", "w");

  // Write result to file
  if (Nt % 2 == 1) {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        fprintf(file, "%f ", *(U + j * M + i));
      }
      fprintf(file, "\n");
    }
  } else {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        fprintf(file, "%f ", *(V + j * M + i));
      }
      fprintf(file, "\n");
    }
  }

  // Close the file
  fclose(file);
  free(U); free(V);
}

In [ ]:
!nvcc Sequential.cu -lm

In [ ]:
!./a.out

In [ ]:
%%writefile CompareOutputs.cu
#include <stdio.h>
#include <stdlib.h>

#define M 100

void readFile(float *data, const char *filename) {
  FILE *file = fopen(filename, "r");
  if (file == NULL) {
    printf("Error opening file %s!\n", filename);
    exit(1);
  }

  for (int j = 0; j < M; j++) {
    for (int i = 0; i < M; i++) {
      if (fscanf(file, "%f", &data[j * M + i]) != 1) {
        printf("Error reading data from file %s!\n", filename);
        fclose(file);
        exit(1);
      }
    }
  }

  fclose(file);
}

int main() {
  float *data1, *data2, *result;
  data1 = (float *)malloc(M * M * sizeof(float));
  data2 = (float *)malloc(M * M * sizeof(float));
  result = (float *)malloc(M * M * sizeof(float));

  // Read data from files
  readFile(data1, "GPU_result.txt");
  readFile(data2, "CPU_result.txt");

  // Subtract corresponding values and print the result
  for (int j = 0; j < M; j++) {
    for (int i = 0; i < M; i++) {
      result[j * M + i] = data1[j * M + i] - data2[j * M + i];
      printf("%f ", result[j * M + i]);
    }
    printf("\n");
  }

  // Free allocated memory
  free(data1);
  free(data2);
  free(result);

  return 0;
}

In [ ]:
!nvcc CompareOutputs.cu -lm

In [ ]:
!./a.out